In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.lines as mlines
from matplotlib.lines import Line2D
import numpy as np
import matplotlib.cm as cm
import xarray as xr
from scipy.stats import ttest_1samp
import matplotlib.ticker as ticker
import pandas as pd
import cftime

# Define the coordinates of the Bay of Bengal region
lon_min, lon_max = 70, 105
lat_min, lat_max = 0, 35
# Define track data for multiple cyclones
cyclone_tracks = [
{
        'name': 'Fani',
        'intensification_range': (23,38),  # Add the rapid intensification range for Cyclone here
        'track': [(2.7, 89.7),(2.7, 89.7), (3.0, 89.4), (3.1, 89.3), (3.2, 89.2), (3.45, 89), (3.7, 88.8), (4.1, 88.8), (4.5, 88.8),
 (4.9, 88.7), (5.2, 88.6), (5.4, 88.5), (5.9, 88.5),(6.3, 88.5), (6.6, 88.2), (6.9, 87.9), (7.3, 87.9), 
 (7.3, 87.9), (7.4, 87.8), (7.7, 87.5), (8.2, 87.0),(8.3, 86.9), (8.4, 86.9), (8.5, 86.9), (8.6, 86.9),
 (8.7, 86.9), (9.2, 86.9), (9.7, 86.8), (10.1, 86.7), (10.4, 86.7), (10.8, 86.6), (11.1, 86.5), (11.7, 86.5),
 (12.3, 86.2), (12.6, 85.7), (13.0, 85.3), (13.3, 84.7), (13.4, 84.5), (13.5, 84.4), (13.6, 84.2), (13.9, 84.0),
 (14.1, 83.9), (14.2, 83.9), (14.5, 84.1), (14.9, 84.1), (15.1, 84.1), (15.2, 84.1), (15.5, 84.2), (15.9, 84.5),
 (16.2, 84.6), (16.7, 84.8), (17.1, 84.8), (17.5, 84.8), (17.8, 84.9), (18.2, 85.0), (18.6, 85.2), (19.1, 85.5),
 (19.6, 85.7), (20.2, 85.9), (20.6, 86.0), (21.1, 86.5), (21.5, 86.7), (21.9, 87.1), (22.5, 87.9), (23.1, 88.2),
 (23.6,  88.8), (24.3, 89.3),(24.75,90.0), (25.2, 90.7),] # Add the track data for Cyclone here
       
    },
{
        'name': 'Amphan',
        'intensification_range': (16,29),  # Add the rapid intensification range for Cyclone here
        'track': [(10.4, 87.0), (10.7, 86.5), (10.9, 86.3), (10.9, 86.3), (10.9, 86.3), (11.0, 86.2), (11.1, 86.1), (11.3, 86.1), 
 (11.4, 86.0), (11.4, 86.0), (11.5, 86.0), (11.7, 86.0), (12.0, 86.0), (12.8, 86.2), (12.5, 86.1), (12.9, 86.4), 
 (13.2, 86.3), (13.3, 86.2), (13.4, 86.2), (13.7, 86.2), (14.0, 86.3), (14.5, 86.4), (14.9, 86.5), (15.2, 86.6), 
 (15.6, 86.7), (16.0, 86.8), (16.5, 86.9), (17.0, 86.9), (17.4, 87.0), (18.1, 87.1), (18.4, 87.2), (18.7, 87.2), 
 (19.1, 87.5), (19.8, 87.7), (20.6, 88.0), (21.4, 88.1), (21.9, 88.4), (22.7, 88.6), (23.3, 89.0), (24.2, 89.0), 
 (24.2, 89.3), (24.7, 89.5), (25.0, 89.6), (25.2,89.6), (25.4, 89.6),] # Add the track data for Cyclone here
       
    },    
{
        'name': 'Mocha',
        'intensification_range': (10,34),  # Add the rapid intensification range for Cyclone here
        'track': [(8.3, 89.5),(8.4,89.4), (8.5, 89.3),(8.5,89.15), (8.5, 89.0), (8.8, 88.9), (9.1, 88.7),(9.55,88.55), (10.0, 88.4),(10.4,88.3), (10.8, 88.2),(11,88.15), (11.2, 88.1),
 (11.4, 88.0), (11.6, 88.0), (11.8, 88.0), (12.2, 88.0), (12.5, 88.1), (12.7, 88.1), (12.9, 88.1), (13.2, 88.1),
 (13.6, 88.2), (14.0, 88.3), (14.3, 88.4), (14.6, 88.6), (14.8, 88.7), (15.1, 88.8), (15.2, 88.9), (15.4, 89.1),
 (15.7, 89.5), (16.0, 90.0), (16.4, 90.3), (16.9, 90.8), (17.4, 90.9), (17.9, 91.0), (18.3, 91.3), (18.7, 91.5),
 (19.3, 91.9), (19.9, 92.5), (20.5, 92.9), (21.1, 93.3), (21.8, 93.8), (22.7, 94.6), (23.5, 95.3), (23.9, 97.8),
 ] # Add the track data for Cyclone here
       
    }, 
{
        'name': 'Mala',
        'intensification_range': (17,32),  # Add the rapid intensification range for Cyclone here
        'track': [(9.5, 90.5), (9.5, 90.5), (9.5, 90.0), (10.0, 89.5), (10.0, 89.5), (10.0, 89.5), (10.0, 89.5), (10.5, 89.0), (10.5, 89.0),
 (11.0, 89.0), (11.0, 89.5), (11.5, 90.0), (12.0, 90.5), (12.0, 90.5), (12.0, 90.5), (12.0, 90.5), (12.5, 90.5), (12.5, 90.5),
 (12.5, 90.5), (13.0, 90.5), (13.0, 90.5), (13.0, 90.5), (13.5, 90.5), (14.0, 91.0), (14.5, 91.5), (15.0, 92.0), (15.3, 92.3), 
 (15.5, 92.5), (16.0, 93.0), (16.0, 93.0), (16.5, 93.5), (16.5, 93.5), (17.0, 94.0), (17.5, 94.5), (18.0, 95.0), (18.5, 95.5),
 (18.5, 95.5), (19.0, 96.0),] # Add the track data for Cyclone here
       
    },
{
        'name': 'Nargis',
        'intensification_range':  (2,10),  # Add the rapid intensification range for Cyclone here
        'track': [(12.0, 87.0), (12.0, 87.0), (12.0, 86.5), (12.0, 86.5), (12.0, 86.5), (12.5, 86.0), (13.0, 85.5), (13.0, 85.5), (13.0, 85.5),
        (13.0, 85.5), (13.0, 85.5), (13.0, 85.5), (13.0, 85.5), (13.0, 85.5), (13.0, 85.5), (13.0, 85.5), (13.5, 85.5), (13.5, 85.5), 
        (13.5, 85.5), (14.0, 85.5), (14.0, 85.5), (14.0, 85.5), (14.0, 86.0), (14.0, 86.0), (14.5, 86.5), (14.5, 86.5), (14.5, 87.0), 
        (14.5, 87.0), (15.0, 87.5), (15.0, 87.5), (15.0, 87.5), (15.5, 88.0), (15.5, 89.0), (16.0, 89.5), (16.0, 90.0), (16.0, 90.5), 
        (16.0, 91.0), (16.0, 91.5), (16.0, 92.0), (16.0, 92.5), (16.0, 93.0), (16.0, 93.5), (16.0, 94.0), (16.0, 94.3), (16.0, 95.0), 
        (16.5, 95.5), (16.5, 95.5), (16.5, 95.5), (17.0, 96.0), (17.5, 96.5), (18.0, 97.0),]
     
   },


    
    # Add more cyclone track data lists as needed
]

# Convert date strings to datetime objects
start_date_amphan = cftime.DatetimeGregorian(2020,5,16)
end_date_amphan = cftime.DatetimeGregorian(2020, 5, 21)
start_date_mocha = cftime.DatetimeGregorian(2023, 5, 9)
end_date_mocha = cftime.DatetimeGregorian(2023, 5, 15)
start_date_mala = cftime.DatetimeGregorian(2006, 4, 25)
end_date_mala = cftime.DatetimeGregorian(2006, 4, 29)
start_date_fani = cftime.DatetimeGregorian(2019, 4, 26)
end_date_fani = cftime.DatetimeGregorian(2019, 5, 4)
start_date_nargis = cftime.DatetimeGregorian(2019, 4, 26)
end_date_nargis = cftime.DatetimeGregorian(2019, 5, 4)
###########################################################################################################
# Load the SST data from the downloaded NetCDF file
sst_data = xr.open_dataset('/home/gokulsuresh/Downloads/sst.day.anom.2020.nc')
sst_data1=sst_data.sel(lat=slice(0,35),lon=slice(70,105))
sst_amphan = sst_data.sel(time=slice(start_date_amphan, end_date_amphan), lat=slice(0, 35), lon=slice(70, 105))
sst_mocha = sst_data.sel(time=slice(start_date_mocha.datetime, end_date_mocha.datetime), lat=slice(0, 35), lon=slice(70, 105))
sst_mala = sst_data.sel(time=slice(start_date_mala.datetime, end_date_mala.datetime), lat=slice(0, 35), lon=slice(70, 105))
sst_fani = sst_data.sel(time=slice(start_date_fani.datetime, end_date_fani.datetime), lat=slice(0, 35), lon=slice(70, 105))
sst_nargis = sst_data.sel(time=slice(start_date_nargis.datetime, end_date_nargis.datetime), lat=slice(0, 35), lon=slice(70, 105))


sst_values = sst_data1['sst'] # Assuming 'sst' is the variable name for sea surface temperature
sst_values1 = sst_amphan['sst'] 
sst_values2 = sst_mocha['sst']
sst_values3 = sst_mala['sst']
sst_values4 = sst_fani['sst']
sst_values5 = sst_nargis['sst']
# Calculate the mean over the time dimension
sst_mean = sst_values.mean(dim='time')
sst_mean1 = sst_values1.mean(dim='time')
sst_mean2 = sst_values2.mean(dim='time')
sst_mean3 = sst_values3.mean(dim='time')
sst_mean4 = sst_values4.mean(dim='time')
sst_mean5 = sst_values5.mean(dim='time')
print(sst_mean)
#print(sst_mean1)
#print(sst_mean2)
#print(sst_mean3)
#print(sst_mean4)
#print(sst_mean5)
sst_anomaly = sst_mean1

# Print the result
#print(sst_anomaly)


################################################################################################################


# Create a new figure
fig = plt.figure(figsize=(10, 10))

# Create a Cartopy map with a PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Set the extent based on the track data
ax.set_extent([lon_min, lon_max, lat_min, lat_max])

# Add grey color to land
ax.add_feature(cfeature.LAND, facecolor='lightgrey')

# Add skyblue color to ocean
#ax.add_feature(cfeature.OCEAN, color='lightblue')
#ax.stock_img()
# Add coastline and borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')


########################################################################################################
# Plot sea surface temperature using contourf
lon, lat = np.meshgrid(sst_anomaly.lon, sst_anomaly.lat)
#levels = np.linspace(-2.5, 2.5, 31) 
sst_contour = ax.contourf(lon, lat, sst_anomaly, transform=ccrs.PlateCarree(), cmap='RdBu_r',levels=30)
# Add colorbar for sea surface temperature

cbar = plt.colorbar(sst_contour, orientation='vertical', fraction=0.02, pad=0.1, aspect=40)
cbar.set_label('SST Anomaly(°C)', fontsize=17)
#########################################################################################################


# Plot all cyclone tracks
for cyclone_data in cyclone_tracks:
    track_data = cyclone_data['track']
    rapid_range = cyclone_data['intensification_range']
    cyclone_name = cyclone_data['name']
    lats, lons = zip(*track_data)
    start_idx, end_idx = rapid_range
    
    for i in range(len(lats) - 1):
        ax.plot(lons[0], lats[0], 'ko', markersize=5, transform=ccrs.PlateCarree())
        gradient = np.linspace(0, 1, len(lons))
        for i, color in enumerate(plt.get_cmap('winter')(gradient)):
             ax.plot(lons[i:i+2], lats[i:i+2], color=color, linewidth=1, transform=ccrs.PlateCarree())
             if start_idx <= i <= end_idx:
              color = 'red'
              ax.plot([lons[i], lons[i+1]], [lats[i], lats[i+1]], color=color,linewidth=1, transform=ccrs.PlateCarree())   
    # Add the cyclone name as text
    ax.text(lons[0], lats[0], cyclone_name, color='black', fontsize=14, ha='left', va='bottom', transform=ccrs.PlateCarree())
# Calculate the number of timesteps for each cyclone
num_timesteps_list = [len(cyclone_data['track']) for cyclone_data in cyclone_tracks]
print(num_timesteps_list)



# Calculate the average number of timesteps
average_num_timesteps = np.mean(num_timesteps_list)
print(average_num_timesteps)
# Calculate the average number of timesteps
average_num_timesteps = int(np.mean(num_timesteps_list))
print(average_num_timesteps)

# List to store mean track data
mean_track_data = []
for timestep in range(average_num_timesteps):  # considering up to timestep 46
    # List to store latitudes and longitudes for the current timestep
    latitudes = []
    longitudes = []

    # Iterate through each cyclone
    for cyclone_data in cyclone_tracks:
        if timestep < len(cyclone_data['track']):
            # Add latitude and longitude for the current timestep
            latitudes.append(cyclone_data['track'][timestep][0])
            longitudes.append(cyclone_data['track'][timestep][1])

    # Calculate mean latitude and longitude for the current timestep
    mean_latitude = np.mean(latitudes)
    mean_longitude = np.mean(longitudes)

    # Add mean track data for the current timestep
    mean_track_data.append((mean_latitude, mean_longitude))

# Plot mean track data
lats, lons = zip(*mean_track_data)
ax.plot(lons, lats, color='black', linestyle='solid', linewidth=2, transform=ccrs.PlateCarree())
ax.plot(lons[0], lats[0], 'ko', markersize=5, transform=ccrs.PlateCarree())



# Legend handles for custom legend entries
ri_line = mlines.Line2D([], [], color='red', linewidth=2, label='RI Phase')
#track_line = Line2D([0], [0], color=cm.viridis(0.7), linewidth=2, label='Evolutionary Track of TC')
genesis_dot = Line2D([0], [0], marker='o', color='black', markersize=10,linestyle='None', label='Genesis Point')
legend_handles = [ri_line,genesis_dot]
ax.legend(handles=legend_handles, loc='upper left', fontsize=12)
# Label latitude and longitude values on the axes
ax.set_xticks(range(lon_min, lon_max + 1, 5), crs=ccrs.PlateCarree())
ax.set_yticks(range(lat_min, lat_max + 1, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
# Add the color bar
cbar = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(0, 1))
cbar.set_array(gradient)
# Add the color bar without specific ticks
cbar = plt.cm.ScalarMappable(cmap=cm.winter, norm=plt.Normalize(0, 1))
cbar.set_array(gradient)
# Set color bar tick parameters without showing numbers
cb = plt.colorbar(cbar, orientation='horizontal', fraction=0.02, pad=0.1, aspect=40)
cb.set_label('Evolution of TC in time', fontsize=17)
cb.ax.set_xticks([])  # Remove tick marks
cb.ax.set_xticklabels([])  # Remove tick labels
# Add custom tick labels
cb.ax.set_xticks([0, 1])  # Set ticks at the beginning and end
cb.ax.set_xticklabels(['Genesis', 'Decay'], fontsize=14)
# Label latitude and longitude
ax.set_xlabel('Longitude',fontsize=22)
ax.set_ylabel('Latitude',fontsize=22)
ax.tick_params(axis='x', labelsize=17)  # Adjust font size for x-axis tick labels
ax.tick_params(axis='y', labelsize=17)  # Adjust font size for y-axis tick labels
# Add minor ticks to both x-axis and y-axis
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.tick_params(axis='x', which='major', width=2,length=6)
plt.tick_params(axis='y', which='major', width=2,length=6)


plt.subplots_adjust(left=0.1, right=0.96, top=.98, bottom=0.1)

plt.savefig('/home/gokulsuresh/Downloads/premonsoon_RI.png',dpi=500)

# Show the plot
plt.show()

ModuleNotFoundError: No module named 'cftime'

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.cm as cm

# Define the coordinates of the Bay of Bengal region
lon_min, lon_max = 70, 105
lat_min, lat_max = 0, 35

# Define track data for multiple cyclones
cyclone_tracks = [
{
        'name': 'Asani',
        'track': [(9.4, 91.3),(9.85,90.85), (10.3, 90.4),(10.55,90.25), (10.8, 90.1),(11.05,89.7), (11.3, 89.3), (11.5, 89.0),
         (11.8, 88.7), (12.0, 88.4), (12.3, 88.2), (12.7, 87.8), (13.1, 87.5), (13.5, 87.1), (13.9, 86.8),
         (14.2, 86.5), (14.6, 86.3), (14.7, 86.0), (14.9, 85.8), (15.0, 85.5), (15.1, 85.2), (15.2, 84.9),
         (15.3, 84.6), (15.4, 84.0), (15.5, 83.5), (15.6, 83.1), (15.7, 82.8), (15.8, 82.4), (15.9, 82.1),
         (16.0, 81.9), (16.1, 81.8), (16.1, 81.6), (16.2, 81.5), (16.2, 81.4), (16.3, 81.4), (16.3, 81.3),
         (16.3, 81.2),(16.25,81.05), (16.2, 80.9),]
    },
{
        'name': 'Yaas',
        'track': [(16.1, 90.2),(16.15,90.05), (16.2, 89.9),(16.25,89.8), (16.3, 89.7), (16.3, 89.7), (16.3, 89.7), 
      (16.5, 89.6), (16.4, 89.6), (16.8, 89.5), (17.1, 89.3), (17.4, 89.2), (17.6, 89.0), (17.8, 88.9),
      (18.0, 88.6), (18.3, 88.3), (18.7, 88.0), (19.1, 88.1), (19.5, 88.0), (19.8, 87.9), (20.1, 87.8), 
      (20.4, 87.6), (20.8, 87.3), (21.2, 87.1), (21.4, 86.9), (21.6, 86.7), (21.8, 86.6), (22.2, 86.2), 
      (22.5, 86.0),(22.65,85.9), (22.8, 85.8), (23.1, 85.7), (23.5, 85.6),(23.9,85.45), (24.3, 85.3),(24.45,85.05), 
      (24.7, 84.8),]
       
    },
{
        'name': 'Mora',
        'track': [(14.0, 88.5), (14.5, 89.5), (15.0, 90.0), (15.4, 90.5), (15.7, 90.7), (16.0, 91.0), (16.3, 91.2),
      (16.6, 91.3), (17.0, 91.3), (17.3, 91.3), (17.8, 91.4), (18.3, 91.5), (18.6, 91.5), (18.8, 91.5), 
      (20.0, 91.6), (20.3, 91.6), (21.1, 91.8), (21.8, 91.9), (22.8, 91.9), (23.6, 92.1), (24.2, 92.2), 
      (24.75,92.3), (25.3, 92.4),]
    },

{
        'name': 'Laila',
        'track': [(10.5,88.5),(10.75,88.25),(11.0,88.0),(11.25,87.75),(11.5,87.5),(11.5,87),(11.5,86.5),(12.0,85.5),(12.5,84.5),(13.0,84.0),(13.0,83.5),
       (13.0,83.0),(13.0,82.5),(13.0,82.0),(13.5,82.0),(13.5,82.0),(13.5,81.5),(14.0,81.5),(14.0,81.5),
       (14.0,81.5),(14.5,81.0),(14.5,81.0),(15.0,81.0),(15.5,80.5),(15.7,80.5),(15.8,80.5),(16.0,80.5),
       (16.0,80.5),(16.0,80.5),(16.0,80.7),(16.2,80.8),(16.5,81.0),(17.0, 81.5),]
     
   },
{
        'name': 'Aila',
        'track': [(16.5, 88.0), (16.5, 88.0), (16.5, 88.0), (16.75, 88.25), (17.0, 88.5), (17.0, 88.5), (17.0, 88.5),
      (18.0, 88.5), (18.0, 88.5), (18.0, 88.5), (18.5, 88.5), (19.0, 88.5), (19.0, 88.5), (20.0, 88.0), 
      (20.0, 88.0), (20.5, 88.0), (21.5, 88.0), (22.0, 88.0), (22.5, 88.0), (23.0, 88.0), (23.5, 88.0), 
      (24.0, 88.0), (25.0, 88.0), (25.5, 88.0), (27.0, 88.5),]

     
   },


    
    # Add more cyclone track data lists as needed
]

###########################################################################################################
# Load the SST data from the downloaded NetCsst_mean1-DF file
sst_data = xr.open_dataset('/home/gokulsuresh/Downloads/SST_climatology.nc')
sst_asani=sst_data.sel(time=slice('2022-5-7','2022-5-12' ))
sst_aila=sst_data.sel(time=slice('2009-5-23','2009-5-26' ))
sst_yaas=sst_data.sel(time=slice('2021-5-23','2021-5-27 '))
sst_mora=sst_data.sel(time=slice('2017-5-28','2017-5-28' ))
sst_laila=sst_data.sel(time=slice('2010-5-17','2010-5-21' ))


sst_values = sst_data['sst'] # Assuming 'sst' is the variable name for sea surface temperature
sst_values1 = sst_asani['sst'] 
sst_values2 = sst_aila['sst']
sst_values3 = sst_yaas['sst']
sst_values4 = sst_mora['sst']
sst_values5 = sst_laila['sst']
print(sst_values1)
# Calculate the mean over the time dimension
sst_mean = sst_values.mean(dim='time')
sst_mean1 = sst_values1.mean(dim='time')
sst_mean2 = sst_values2.mean(dim='time')
sst_mean3 = sst_values3.mean(dim='time')
sst_mean4 = sst_values4.mean(dim='time')
sst_mean5 = sst_values5.mean(dim='time')
sst_anomaly = np.mean([sst_mean1],[sst_mean2],[sst_mean3],[sst_mean4],[sst_mean5], axis=0) - sst_mean

# Print the result
print(sst_anomaly)


################################################################################################################


# Create a new figure
fig = plt.figure(figsize=(10, 10))

# Create a Cartopy map with a PlateCarree projection
ax = plt.axes(projection=ccrs.PlateCarree())

# Set the extent based on the track data
ax.set_extent([lon_min, lon_max, lat_min, lat_max])

# Add grey color to land
ax.add_feature(cfeature.LAND, facecolor='lightgrey')

# Add skyblue color to ocean
#ax.add_feature(cfeature.OCEAN, color='lightblue')
#ax.stock_img()
# Add coastline and borders
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')


for cyclone_data in cyclone_tracks:
    track_data = cyclone_data['track']
    cyclone_name = cyclone_data['name']

    lats, lons = zip(*track_data)

    gradient = np.linspace(0, 1, len(lons))
    for i, color in enumerate(plt.get_cmap('winter')(gradient)):
      ax.plot(lons[i:i+2], lats[i:i+2], color=color, linewidth=1.4, transform=ccrs.PlateCarree())

    # Mark the starting point
    ax.plot(lons[0], lats[0], 'ko', markersize=5, color='black', transform=ccrs.PlateCarree())

    # Add the cyclone name as text
    ax.text(lons[0], lats[0], cyclone_name, color='black', fontsize=14, ha='left', va='bottom', transform=ccrs.PlateCarree())
 # Calculate the number of timesteps for each cyclone
num_timesteps_list = [len(cyclone_data['track']) for cyclone_data in cyclone_tracks]
print(num_timesteps_list)

# Calculate the average number of timesteps
average_num_timesteps = np.mean(num_timesteps_list)
print(average_num_timesteps)


# Calculate the average number of timesteps
average_num_timesteps = int(np.mean(num_timesteps_list))
print(average_num_timesteps)

# List to store mean track data
mean_track_data = []

for timestep in range(average_num_timesteps):  # considering up to timestep 46
    # List to store latitudes and longitudes for the current timestep
    latitudes = []
    longitudes = []

    # Iterate through each cyclone
    for cyclone_data in cyclone_tracks:
        if timestep < len(cyclone_data['track']):
            # Add latitude and longitude for the current timestep
            latitudes.append(cyclone_data['track'][timestep][0])
            longitudes.append(cyclone_data['track'][timestep][1])

    # Calculate mean latitude and longitude for the current timestep
    mean_latitude = np.mean(latitudes)
    mean_longitude = np.mean(longitudes)

    # Add mean track data for the current timestep
    mean_track_data.append((mean_latitude, mean_longitude))

# Plot mean track data
lats, lons = zip(*mean_track_data)
ax.plot(lons, lats, color='black', linestyle='solid', linewidth=2, transform=ccrs.PlateCarree())
ax.plot(lons[0], lats[0], 'ko', markersize=5, transform=ccrs.PlateCarree())
  
# Add the color bar
cbar = plt.cm.ScalarMappable(cmap=cm.viridis, norm=plt.Normalize(0, 1))
cbar.set_array(gradient)

# Add the color bar without specific ticks
cbar = plt.cm.ScalarMappable(cmap=cm.winter, norm=plt.Normalize(0, 1))
cbar.set_array(gradient)

# Set color bar tick parameters without showing numbers
cb = plt.colorbar(cbar, orientation='horizontal', fraction=0.01, pad=0.1, aspect=40)
cb.set_label('Evolution of TC in time', fontsize=17)
cb.ax.set_xticks([])  # Remove tick marks
cb.ax.set_xticklabels([])  # Remove tick labels
# Add custom tick labels
cb.ax.set_xticks([0, 1])  # Set ticks at the beginning and end
cb.ax.set_xticklabels(['Genesis', 'Decay'], fontsize=14)

# Legend handles for custom legend entries
#track_line = Line2D([0], [0], color=cm.viridis(0.7), linewidth=2, label='Evolutionary Track of TC')
genesis_dot = Line2D([0], [0], marker='o', color='black', markersize=10,linestyle='None', label='Genesis Point')
legend_handles = [ genesis_dot]
ax.legend(handles=legend_handles, loc='upper left', fontsize=12)
# Label latitude and longitude values on the axes
ax.set_xticks(range(lon_min, lon_max + 1, 5), crs=ccrs.PlateCarree())
ax.set_yticks(range(lat_min, lat_max + 1, 5), crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
########################################################################################################
# Plot sea surface temperature using contourf
lon, lat = np.meshgrid(sst_anomaly.longitude, sst_anomaly.latitude)
levels = np.linspace(-3, 3, 31) 
sst_contour = ax.contourf(lon, lat, sst_anomaly, transform=ccrs.PlateCarree(), cmap='RdBu_r',levels=levels)
# Add colorbar for sea surface temperature

cbar = plt.colorbar(sst_contour, orientation='vertical', fraction=0.02, pad=0.1, aspect=40)
cbar.set_label('SST Anomaly(°C)', fontsize=17)
#########################################################################################################

# Label latitude and longitude
ax.set_xlabel('Longitude',fontsize=22)
ax.set_ylabel('Latitude',fontsize=22)
ax.tick_params(axis='x', labelsize=17)  # Adjust font size for x-axis tick labels
ax.tick_params(axis='y', labelsize=17)  # Adjust font size for y-axis tick labels
# Add minor ticks to both x-axis and y-axis
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
plt.tick_params(axis='x', which='major', width=2,length=6)
plt.tick_params(axis='y', which='major', width=2,length=6)

plt.subplots_adjust(left=0.1, right=0.96, top=.98, bottom=0.1)

plt.savefig('/home/gokulsuresh/Downloads/premonsoon_nRI.png',dpi=300)
# Show the plot
plt.show()

<xarray.DataArray 'sst' (time: 48, expver: 2, latitude: 141, longitude: 141)>
[1908576 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 70.0 70.25 70.5 70.75 ... 104.5 104.8 105.0
  * latitude   (latitude) float32 35.0 34.75 34.5 34.25 ... 0.75 0.5 0.25 0.0
  * expver     (expver) int32 1 5
  * time       (time) datetime64[ns] 2022-05-07 ... 2022-05-12T21:00:00
Attributes:
    units:      K
    long_name:  Sea surface temperature


: 